In [1]:
import xarray as xr
import numpy as np
from numpy import pi, sin, cos, arccos, clip, deg2rad
import numpy.ma as ma
from datetime import datetime
import dask
import time
import zarr

#### Currently 30 equally structured datasets stored in 30 zarr-stores.
##### All dataset has the same length of tabular data,
All 30 datasets has 5 columns,  `latitude`, `longitude`, `time`, `mean`, `anomalies`, each with shape = (2105319,), where mean and anomalies is the only value different between the datasets.

In [36]:
import json
import gcsfs

with open('pangeo-181919-e7bc5bdaf4d5.json') as f:
    token = json.load(f)
gcs = gcsfs.GCSFileSystem(token=token)

plevel = 4 # Choosing an arbitrary dataset out of the 30
dspath='pangeo-argo-eke/global/mean_and_anomalies/with_pressure_coordinate/readable_ws_and_NHarm/mean_and_anomalies_global_ws500_plevel'+str(plevel)+'.zarr'
mapper_alt0 = gcs.get_mapper(dspath)
d = xr.open_zarr(mapper_alt0, consolidated=True, decode_cf=False )

In [28]:
#import fsspec.utils
#fsspec.utils.setup_logging(logger_name="gcsfs")

### Storing

In [37]:
# Mean and anomalies for pressurelevel is saved to output-path:
ds = xr.Dataset(
    data_vars = dict(
        Mean = ( ["i", "pressure"], d.Mean.load().values ),
        Anomalies = ( ["i", "pressure"], d.Anomalies.load().values ),
    ),
    coords=dict(
        time = ( ["i"], d.time.load().values ),
        latitude = ( ["i"], d.latitude.load().values ),
        longitude = ( ["i"], d.longitude.load().values ),
        pressure = (["pressure"], d.pressure.load().values ),
    ),
    attrs=dict(
        description = 'Estimated mean dynamic height on profile-coordinates, and anomalies by subtracting estimated mean from observations',
        pressureindex = d.pressureindex,
        number_of_harmonics = d.number_of_harmonics,
        window_size = d.window_size,
        creation_date = str( datetime.now() )
    ),
)

ds.time.attrs["standard_name"] = 'time'
ds.time.attrs["units"] = 'days since 1970-01-01 00:00:00'
ds.latitude.attrs["standard_name"] = 'latitude'
ds.latitude.attrs["units"] = 'degrees_north'
ds.longitude.attrs["standard_name"] = 'longitude'
ds.longitude.attrs["units"] = 'degrees_east'
ds.pressure.attrs["standard_name"] = 'pressure'
ds.pressure.attrs["units"] = 'decibar'
ds.Mean.attrs["standard_name"] = 'Estimated mean dynamic height'
ds.Mean.attrs["units"] = 'm^2/s^2'
ds.Anomalies.attrs["standard_name"] = 'dynamic height anomalies'
ds.Anomalies.attrs["units"] = 'm^2/s^2'

dsc = ds.chunk()
outfile = 'pangeo-argo-eke/chunk_alternatives/global_mean_and_anomalies_plevel'+str(plevel)+'_current_zarr-storing-code.zarr'
mapper_alt1 = gcs.get_mapper(outfile)
dsc.to_zarr(mapper_alt1, consolidated=True)

In [42]:
# Mean and anomalies for pressurelevel is saved to output-path:
ds = xr.Dataset(
    data_vars = dict(
        Mean = ( ["i", "pressure"], d.Mean.load().values ),
        Anomalies = ( ["i", "pressure"], d.Anomalies.load().values ),
    ),
    coords=dict(
        time = ( ["i"], d.time.load().values ),
        latitude = ( ["i"], d.latitude.load().values ),
        longitude = ( ["i"], d.longitude.load().values ),
        pressure = (["pressure"], d.pressure.load().values ),
    ),
    attrs=dict(
        description = 'Estimated mean dynamic height on profile-coordinates, and anomalies by subtracting estimated mean from observations',
        pressureindex = d.pressureindex,
        number_of_harmonics = d.number_of_harmonics,
        window_size = d.window_size,
        creation_date = str( datetime.now() )
    ),
)

ds.time.attrs["standard_name"] = 'time'
#ds.time.attrs["units"] = 'days since 1970-01-01 00:00:00'
ds.latitude.attrs["standard_name"] = 'latitude'
ds.latitude.attrs["units"] = 'degrees_north'
ds.longitude.attrs["standard_name"] = 'longitude'
ds.longitude.attrs["units"] = 'degrees_east'
ds.pressure.attrs["standard_name"] = 'pressure'
ds.pressure.attrs["units"] = 'decibar'
ds.Mean.attrs["standard_name"] = 'Estimated mean dynamic height'
ds.Mean.attrs["units"] = 'm^2/s^2'
ds.Anomalies.attrs["standard_name"] = 'dynamic height anomalies'
ds.Anomalies.attrs["units"] = 'm^2/s^2'

dsc = ds.chunk()
outfile = 'pangeo-argo-eke/chunk_alternatives/global_mean_and_anomalies_plevel'+str(plevel)+'_time-dtype-float64.zarr'
mapper_alt2 = gcs.get_mapper(outfile)
dsc.to_zarr(mapper_alt2, consolidated=True)

#### Size of each dataset in megabytes

In [46]:
dsc.nbytes/1e6

84.212768

### Connect to a dask-cluster and set cluster-options

In [47]:
from dask_gateway import GatewayCluster, Gateway
from distributed import Client

g = Gateway()
g.list_clusters()

[]

In [48]:
#cluster = g.connect(g.list_clusters()[0].name)

In [49]:
options = g.cluster_options()
options.worker_cores = 2; options.worker_memory = 4
# Create a cluster with those options
cluster = g.new_cluster(options)

In [50]:
#g.list_clusters()

In [51]:
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: dask_gateway.GatewayCluster
Dashboard: /services/dask-gateway/clusters/prod.3d80ce07c5574c1881a4e47e530d5240/status,


In [52]:
cluster.scale(1)

### Loading of data from cloud-storage<br>
Want to load 4 columns of data, `latitude`, `longitude`, `time`, `anomalies`, into memory and do a computation. Which is done embarrasingly parallel ( x 1e6  ).

Loading data only:

In [123]:
@dask.delayed
def load_chunksauto(mapper):
    """Load data from zarr-store into worker-memory"""
    
    ds = xr.open_zarr(mapper, consolidated=True, chunks='auto') 
    data0 = ds.Anomalies.sel(pressure=50).load().values
    ii = ~xr.apply_ufunc(np.isnan, data0)
    data, lat, lon, time = data0[ii], ds.latitude.load().values[ii], ds.longitude.load().values[ii], ds.time.load().values[ii]
    
    # Calculation using data-, lat-, lon-, and time-arrays
    #
    #
    #
    return None

In [124]:
@dask.delayed
def load_chunksNone(mapper):
    """Load data from zarr-store into worker-memory"""
    
    ds = xr.open_zarr(mapper, consolidated=True, chunks=None) 
    data0 = ds.Anomalies.sel(pressure=50).values
    ii = ~xr.apply_ufunc(np.isnan, data0)
    data, lat, lon, time = data0[ii], ds.latitude.values[ii], ds.longitude.values[ii], ds.time.values[ii]
    
    # Calculation using data-, lat-, lon-, and time-arrays
    #
    #
    #
    return None

In [125]:
@dask.delayed
def zarr_open(mapper):
    """Load data from zarr-store into worker-memory"""
    
    z = zarr.open_consolidated(mapper)
    data0 = np.squeeze(z.Anomalies[:])
    ii = ~np.isnan(data0)
    data, lat, lon, time = data0[ii], z.latitude[:][ii], z.longitude[:][ii], z.time[:][ii]
    
    # Calculation using data-, lat-, lon-, and time-arrays
    #
    #
    #
    return None

In [126]:
@dask.delayed
def zarr_convenience_open(mapper):
    """Load data from zarr-store into worker-memory"""
    
    z = zarr.convenience.open_consolidated(mapper)
    data0 = np.squeeze(z.Anomalies[:])
    ii = ~np.isnan(data0)
    data, lat, lon, time = data0[ii], z.latitude[:][ii], z.longitude[:][ii], z.time[:][ii]
    
    # Calculation using data-, lat-, lon-, and time-arrays
    #
    #
    #
    return None

### Timing loading of data:<br>

<br>
ds = xr.open_zarr(mapper, consolidated=True, chunks='auto'); ...  

In [127]:
%timeit dask.compute( load_chunksauto(mapper_alt1) )[0]

1.47 s ± 92.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [128]:
%timeit dask.compute( load_chunksauto(mapper_alt2) )[0]

1.01 s ± 90.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


<br>
ds = xr.open_zarr(mapper, consolidated=True, chunks=None); ...

In [129]:
%timeit dask.compute( load_chunksNone(mapper_alt1) )[0]

1.61 s ± 84.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [130]:
%timeit dask.compute( load_chunksNone(mapper_alt2) )[0]

1.29 s ± 107 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


<br>
z = zarr.open_consolidated(mapper); ... 

In [131]:
%timeit dask.compute( zarr_open(mapper_alt1) )[0]

1.05 s ± 34.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [132]:
%timeit dask.compute( zarr_open(mapper_alt2) )[0]

1.3 s ± 106 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


<br>
z = zarr.convenience.open_consolidated(mapper); ... 

In [133]:
%timeit dask.compute( zarr_convenience_open(mapper_alt1) )[0]

1.05 s ± 31.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [134]:
%timeit dask.compute( zarr_convenience_open(mapper_alt2) )[0]

1.37 s ± 178 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### Scaling down and closing cluster

In [135]:
cluster.scale(0)

In [136]:
cluster.close()

In [137]:
cluster.shutdown()

2023-07-27 15:17:46,375 - distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client
